In [ ]:
 from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

from PIL import Image

In [ ]:
#8 MIN TO RUN
def load_data():
    # Load training images and labels
    train_images = []
    train_labels = []
    train_dir = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/train/images'
    train_label_file = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/train/labels'
    for filename in os.listdir(train_dir):
        img = Image.open(os.path.join(train_dir, filename))
        img = img.resize((224, 224)) # Resize to a common size
        img_arr = np.array(img)
        train_images.append(img_arr)

        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_file_path = os.path.join(train_label_file, label_filename)
        with open(label_file_path, 'r') as f:
            text = f.read().strip()
            # Map "no_mask" to 0, "mask" to 1, and "improper_mask" to 2
            if text == "1":
                label = 1
            elif text == "0":
                label = 0
            elif text == "2":
                label = 2
            train_labels.append(label)

    # Load testing images and labels
    test_images = []
    test_labels = []
    test_dir = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/test/images'
    test_label_dir = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/test/labels'
    for filename in os.listdir(test_dir):
        img = Image.open(os.path.join(test_dir, filename))
        img = img.resize((224, 224)) # Resize to a common size
        img_arr = np.array(img)
        test_images.append(img_arr)

        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_file_path = os.path.join(test_label_dir, label_filename)
        with open(label_file_path, 'r') as f:
            text = f.read().strip()
            # Map "no_mask" to 0, "mask" to 1, and "improper_mask" to 2
            if text == "1":
                label = 1
            elif text == "0":
                label = 0
            elif text == "2":
                label = 2
            test_labels.append(label)

    # Convert the lists to numpy arrays
    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)

    return train_images, test_images, train_labels, test_labels

# Load the dataset
X_train, X_test, y_train, y_test = load_data()

print("Number of training images: ", len(X_train))
print("Number of training labels: ", len(y_train))
print("Number of testing images: ", len(X_test))
print("Number of testing labels: ", len(y_test))

print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_test: ", y_test.shape)

# Print information about the dataset
print("Number of training images: ", X_train.shape[0])
print("Number of training labels: ", y_train.shape[0])
print("Number of testing images: ", X_test.shape[0])
print("Number of testing labels: ", y_test.shape[0])

Number of training images:  2394
Number of training labels:  2394
Number of testing images:  458
Number of testing labels:  458
Shape of X_train:  (2394, 224, 224, 3)
Shape of y_train:  (2394,)
Shape of X_test:  (458, 224, 224, 3)
Shape of y_test:  (458,)
Number of training images:  2394
Number of training labels:  2394
Number of testing images:  458
Number of testing labels:  458


In [ ]:
# 12 MIN to run
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix

# Normalize the data
X_train_normalized = X_train / 255.0
X_test_normalized = X_test / 255.0

# One-hot encode the labels
y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes=3)

# Create the CNN model with reduced capacity
model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model with reduced batch size
history = model.fit(X_train_normalized, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test_normalized, y_test_one_hot))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_normalized, y_test_one_hot, verbose=2)
print('\nTest accuracy:', test_acc)

# Calculate F1 score, recall, precision, and confusion matrix
y_pred = np.argmax(model.predict(X_test_normalized), axis=-1)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

print("\nF1 Score: ", f1)
print("Recall: ", recall)
print("Precision: ", precision)
print("Confusion Matrix:\n", conf_matrix)

Epoch 1/10
75/75 [==============================] - 125s 2s/step - loss: 0.4862 - accuracy: 0.8634 - val_loss: 0.2597 - val_accuracy: 0.9236
Epoch 2/10
75/75 [==============================] - 126s 2s/step - loss: 0.2208 - accuracy: 0.9336 - val_loss: 0.2238 - val_accuracy: 0.9214
Epoch 3/10
75/75 [==============================] - 125s 2s/step - loss: 0.2073 - accuracy: 0.9361 - val_loss: 0.1815 - val_accuracy: 0.9279
Epoch 4/10
75/75 [==============================] - 125s 2s/step - loss: 0.1860 - accuracy: 0.9407 - val_loss: 0.1852 - val_accuracy: 0.9279
Epoch 5/10
75/75 [==============================] - 128s 2s/step - loss: 0.1613 - accuracy: 0.9449 - val_loss: 0.2025 - val_accuracy: 0.9323
Epoch 6/10
75/75 [==============================] - 127s 2s/step - loss: 0.1481 - accuracy: 0.9532 - val_loss: 0.1872 - val_accuracy: 0.9279
Epoch 7/10
75/75 [==============================] - 125s 2s/step - loss: 0.1502 - accuracy: 0.9495 - val_loss: 0.1858 - val_accuracy: 0.9410
Epoch 8/10
75

In [ ]:
import joblib
import os

# Define directory path to save the joblib file
directory = '/content/drive/MyDrive/CW_Folder_UG/Models'

# Create directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Define file path to save the joblib file
filepath = os.path.join(directory, 'cnn.joblib')

# Dump the object to the joblib file
joblib.dump(model, filepath)

['/content/drive/MyDrive/CW_Folder_UG/Models/cnn.joblib']

In [ ]:
import joblib
import os

# create folder if it doesn't exist
folder_path = os.path.join('content', 'drive', 'MyDrive', 'CW_Folder_UG', 'Models')
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# save model in folder
file_path = os.path.join(folder_path, 'cnn.joblib')
joblib.dump(model, file_path)


['content/drive/MyDrive/CW_Folder_UG/Models/cnn.joblib']